<a href="https://colab.research.google.com/github/andreidore/aicrowd_blitz_may_2020/blob/master/pkhnd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 112kB 12.7MB/s 
     |████████████████████████████████| 460kB 16.7MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=1a28bf21f580737996f08b72de68561f95741ccf100163210ef345e85435521b
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=e2a5457573500354a57a915aafddcbe970a5028e54d28cb6d7b03d976ccb3779
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=d21a

In [2]:
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/test.csv
!wget https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/train.zip
!unzip train.zip

--2020-05-10 20:25:13--  https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/test.csv
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.10, 130.117.252.13, 130.117.252.12, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 538694 (526K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>] 526.07K  1.12MB/s    in 0.5s    

2020-05-10 20:25:14 (1.12 MB/s) - ‘test.csv’ saved [538694/538694]

--2020-05-10 20:25:16--  https://s3.eu-central-1.wasabisys.com/aicrowd-public-datasets/aicrowd_educational_pkhnd/data/public/train.zip
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.16, 130.117.252.12, 130.117.252.13, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.16|:443... connected.
HTTP 

In [0]:
import os
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
import xgboost as xgb


In [4]:
pd.__version__

'1.0.3'

In [0]:
train_data_path = "train.csv" #path where data is stored
train_data = pd.read_csv(train_data_path) #load data in dataframe using pandas

In [6]:
print(train_data.shape)
print(train_data.head())

(1000000, 11)
   S1  C1  S2  C2  S3  C3  S4  C4  S5  C5  label
0   1   1   1  13   2   4   2   3   1  12      0
1   3  12   3   2   3  11   4   5   2   5      1
2   1   9   4   6   1   4   3   2   3   9      1
3   1   4   3  13   2  13   2   1   3   6      1
4   3  10   2   7   1   2   2  11   4   9      0


In [7]:
X,y = train_data.iloc[:,:-1],train_data.iloc[:,-1]
print(X.shape)
print(X.head())
print(y.shape)
print(y.head())

(1000000, 10)
   S1  C1  S2  C2  S3  C3  S4  C4  S5  C5
0   1   1   1  13   2   4   2   3   1  12
1   3  12   3   2   3  11   4   5   2   5
2   1   9   4   6   1   4   3   2   3   9
3   1   4   3  13   2  13   2   1   3   6
4   3  10   2   7   1   2   2  11   4   9
(1000000,)
0    0
1    1
2    1
3    1
4    0
Name: label, dtype: int64


In [8]:
enc = OneHotEncoder(sparse=False)
enc.fit(X)
X = enc.transform(X)
X=pd.DataFrame(X)
print(X.shape)
print(X.head())

(1000000, 85)
    0    1    2    3    4    5    6   ...   78   79   80   81   82   83   84
0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  1.0  0.0
1  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0
3  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0

[5 rows x 85 columns]


In [0]:
#pandas_profiling.ProfileReport(train_data)

In [10]:
X_train, X_val,y_train,y_val= train_test_split(X,y, test_size=0.2, random_state=42)
print(X_train)
print(y_train)
print(X_val)
print(y_val)

         0    1    2    3    4    5    6   ...   78   79   80   81   82   83   84
566853  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
382311  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
241519  0.0  0.0  0.0  1.0  0.0  1.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
719220  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0
905718  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  1.0  0.0
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
259178  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  1.0  0.0
365838  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
131932  0.0  1.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
671155  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0
121958  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[800000 rows x 

In [11]:
#classifier = SVC(gamma='auto',max_iter=10)
#classifier = DecisionTreeClassifier(random_state=42, max_depth=100)
#classifier = xgb.XGBClassifier(random_state=42)
#classifier.fit(X_train, y_train,early_stopping_rounds=10,eval_set=[(X_val, y_val)])
classifier = RandomForestClassifier(random_state=42,n_estimators=100)
classifier.fit(X_train, y_train)
#

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [12]:
y_pred = classifier.predict(X_val)
precision = precision_score(y_val,y_pred,average='micro')
recall = recall_score(y_val,y_pred,average='micro')
accuracy = accuracy_score(y_val,y_pred)
f1 = f1_score(y_val,y_pred,average='macro')
print("Accuracy of the model is :" ,accuracy)
print("Recall of the model is :" ,recall)
print("Precision of the model is :" ,precision)
print("F1 score of the model is :" ,f1)

Accuracy of the model is : 0.91701
Recall of the model is : 0.91701
Precision of the model is : 0.91701
F1 score of the model is : 0.25354103260013583


## Submission

In [17]:
final_test_path = "test.csv"
final_test = pd.read_csv(final_test_path)

enc = OneHotEncoder(sparse=False)
enc.fit(final_test)
final_test = enc.transform(final_test)
final_test=pd.DataFrame(final_test)
print(X.shape)
print(X.head())


submission = classifier.predict(final_test)

submission = pd.DataFrame(submission)
submission.to_csv('/tmp/submission.csv',header=['label'],index=False)
from google.colab import files
files.download('/tmp/submission.csv')

(1000000, 85)
    0    1    2    3    4    5    6   ...   78   79   80   81   82   83   84
0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  1.0  0.0
1  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0
3  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0

[5 rows x 85 columns]
